<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Fasttext1_Bin_Not_Loading_Error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode
!pip install word2number
!pip install contractions

  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=beff3aea39d80836f467cac9c7d0b1a0d2525996dd51696290e490707b2583f5
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
     |████████████████████████████████| 284 kB 8.1 MB/s 
     |████████████████████████████████| 321 kB 66.8 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=ac55d29d01d703d087d88315a9bf7a8bafba6aa7f47f095315e1fe5af53cd6e6
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1

df_fake.count()


id           5323
news_url     5067
title        5323
tweet_ids    5135
labelML      5323
dtype: int64

In [ ]:
#df= df_fake.head(50).append(df_real.head(50))
df= df_fake.append(df_real)
df.reset_index(drop=True, inplace=True)

In [ ]:
print(df.columns)
df.count()
print(df)

Index(['id', 'news_url', 'title', 'tweet_ids', 'labelML'], dtype='object')
                         id  ... labelML
0      gossipcop-2493749932  ...       0
1      gossipcop-4580247171  ...       0
2       gossipcop-941805037  ...       0
3      gossipcop-2547891536  ...       0
4      gossipcop-5476631226  ...       0
...                     ...  ...     ...
22135      gossipcop-891749  ...       1
22136      gossipcop-941486  ...       1
22137      gossipcop-953143  ...       1
22138      gossipcop-954366  ...       1
22139      gossipcop-877062  ...       1

[22140 rows x 5 columns]


In [ ]:
print(df)

                         id  ... labelML
0      gossipcop-2493749932  ...       0
1      gossipcop-4580247171  ...       0
2       gossipcop-941805037  ...       0
3      gossipcop-2547891536  ...       0
4      gossipcop-5476631226  ...       0
...                     ...  ...     ...
22135      gossipcop-891749  ...       1
22136      gossipcop-941486  ...       1
22137      gossipcop-953143  ...       1
22138      gossipcop-954366  ...       1
22139      gossipcop-877062  ...       1

[22140 rows x 5 columns]


In [ ]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)
df1.count()

title      22140
labelML    22140
dtype: int64

In [ ]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [ ]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
#---------df1.iloc[:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__' + x)
#print(df1)

In [ ]:
#print(df1)
df1.count()

title      22140
labelML    22140
dtype: int64


 #df = df.loc[:,"title"]
 # just for loc practice, ignore.

 X_train - This includes your all independent variables,these will be used to train the model, also as we have specified the test_size = 0.4, this means 60% of observations from your complete data will be used to train/fit the model and rest 40% will be used to test the model.

2). X_test - This is remaining 40% portion of the independent variables from the data which will not be used in the training phase and will be used to make predictions to test the accuracy of the model.

3). y_train - This is your dependent variable which needs to be predicted by this model, this includes category labels against your independent variables, we need to specify our dependent variable while training/fitting the model.

4). y_test - This data has category labels for your test data, these labels will be used to test the accuracy between actual and predicted categories.


In [ ]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
df1.iloc[:5322, 0] = df1.iloc[:, 0].apply(lambda x: '__label__0 ' + x)
df1.iloc[5323:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__1 ' + x)

# NOTE BIG NOTE In general it is important to properly preprocess your data. In particular our example scripts in the root folder do this. i can see why the last cell is wierd, fix in post.


In [ ]:
print(df1)

                                                   title  labelML
0      __label__0 did miley cyrus and liam hemsworth ...        0
1      __label__0 paris jackson cara delevingne enjoy...        0
2      __label__0 celebrities join tax march in prote...        0
3      __label__0 cindy crawford daughter kaia gerber...        0
4      __label__0 full list of oscar nominations variety        0
...                                                  ...      ...
22135  __label__1 hollywood film awards the complete ...        1
22136  __label__1 jada pinkett smith explains why son...        1
22137  __label__1 tinsley mortimer reacts to luann de...        1
22138  __label__1 prince harry carries on princess di...        1
22139  __label__1 kylie jenner is actually terrified ...        1

[22140 rows x 2 columns]


In [ ]:
df1.count()

title      22140
labelML    22140
dtype: int64

In [ ]:
df1.reset_index(drop=True, inplace=True)
X, y = df1.iloc[:, 1:], df1['labelML']

# Create the training and test sets #IMP: use stratify when dataset is not balanced 
#https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas 
#--for few entries-- #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 1.000000


In [ ]:
df2=df1
# ------------- df2 = df2.reindex(columns=['labelML', 'title'])
#print(df1)

In [ ]:
traindf, testdf = train_test_split(df2, test_size=0.2)
print(traindf)
print(testdf)  

traindf.count()

                                                   title  labelML
11016  __label__1 simon cowell rushed to hospital aft...        1
5807   __label__1 busy philipps responds to backlash ...        1
17607  __label__1 real housewives of potomac star mon...        1
4869   __label__0 star girly princess suri cruise vs ...        0
11580  __label__1 johnny depp star in crisis and the ...        1
...                                                  ...      ...
15189                       __label__1 we day california        1
20471  __label__1 kevin spacey replaced by christophe...        1
2360   __label__0 is brad pitt body odor becoming pro...        0
14326  __label__1 have adam sandler drew barrymore ev...        1
9655   __label__1 kelly clarkson responds to grammys ...        1

[17712 rows x 2 columns]
                                                   title  labelML
5045   __label__0 katy perry and orlando bloom cozy u...        0
17950  __label__1 cancer destroyed tristan butt bu

title      17712
labelML    17712
dtype: int64

In [ ]:
# Saving the CSV file as a text file to train/test the classifier
import csv #needed
traindf[['labelML', 'title']].to_csv('traindf.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

testdf[['labelML', 'title']].to_csv('testdf.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [ ]:
!pip install fasttext
import fasttext
# Training the fastText classifier
model = fasttext.train_supervised('traindf.txt', wordNgrams = 2)

# Evaluating performance on the entire test file
model.test('testdf.txt')                      

# Predicting on a single input
testdf['labelML'] = testdf['labelML'].astype(str)
model.predict(testdf.iloc[56, 0])
#model.test(testdf.txt)

# Save the trained model
#model.save_model('model.bin')

(('__label__1',), array([0.84595311]))

In [ ]:
testdf.head(10)
#print(testdf.columns)

,title,labelML
5045,__label__0 katy perry and orlando bloom cozy u...,0
17950,__label__1 cancer destroyed tristan butt but w...,1
3130,__label__0 britney spears speaks pop star talk...,0
9167,__label__1 star wars fans show up for solo red...,1
2478,__label__0 blake shelton reveals that he misse...,0
16454,__label__1 vanderpump rules ariana madix recal...,1
11653,__label__1 robert henstridge,1
15666,__label__1 see every celebrity at the royal we...,1
19586,__label__1 meghan markle dad thomas causing ro...,1
1802,__label__0 selena gomez and justin bieber hang...,0


This model is running, so trying to change with vectors, or textual input for new vectors. Accuracy is only 50 percent coz I've given it such a balanced dataset.

In [ ]:
# Predicting on a single input
#model.predict(test.iloc['Miley cyrus is new'])
#model.test(testdf.txt)
#use this later when we have the representations I guess. 


In [ ]:
# Once the model is trained, we can retrieve the list of words and labels

# model.words 
model.labels
#It returns all words in the vocabulary, sorted by decreasing frequency. 

['__label__1', '__label__0']

In [ ]:
# To evaluate our model by computing the precision at 1 (P@1) and the recall on a test set, we use the test function:

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('testdf.txt'))

N	4428
P@1	1.000
R@1	1.000


In [ ]:
#We can also predict labels for a specific text. By default, predict returns only one label :
# the one with the highest probability. You can also predict more than one label by specifying the parameter k:
model.predict("poo mooo op", k=2)

(('__label__1', '__label__0'), array([0.90478355, 0.09523643]))

In [ ]:
#traindf['title'] = 
model.get_word_vector("the")

array([ 0.02137857,  0.01676384,  0.04503896, -0.0524342 ,  0.06966472,
        0.02554192, -0.02811272, -0.0125381 ,  0.03379885,  0.022515  ,
       -0.00410887, -0.043385  ,  0.00548688,  0.01112284, -0.05795665,
       -0.02040619,  0.03973591,  0.05103175, -0.05173127, -0.02768937,
       -0.01450215,  0.06939985,  0.02702775,  0.00922399, -0.01715516,
       -0.07961302, -0.03875691, -0.03920429, -0.00676623,  0.02291585,
       -0.04950753, -0.02018066, -0.02099022, -0.07745673, -0.09214057,
       -0.02726264, -0.0315864 ,  0.0202981 ,  0.0233455 , -0.03360463,
       -0.04834947,  0.10499092, -0.00102818,  0.05098762,  0.05997887,
        0.01068992,  0.05072624, -0.01362669, -0.04072247,  0.0327213 ,
       -0.03744594,  0.03770958, -0.02543122,  0.06992245, -0.07166345,
        0.03161179, -0.05292026, -0.00376694,  0.04681172, -0.00822764,
        0.0037593 ,  0.05232479,  0.07301264,  0.10612043,  0.01843838,
       -0.00348117, -0.08782569,  0.00184562, -0.01502951, -0.02

In [ ]:
model.get_sentence_vector("simon cowell rushed to hospital")

array([ 0.00081413,  0.00415767,  0.0056617 , -0.00708038,  0.00977401,
        0.00481745, -0.00278039, -0.00195706,  0.00271939,  0.00450671,
        0.00133211, -0.00539508,  0.00103059,  0.00293263, -0.0096593 ,
       -0.00487769,  0.00284352,  0.00556662, -0.00440305, -0.00490769,
       -0.00049042,  0.01025262,  0.00385767, -0.00153705, -0.00193012,
       -0.01131517, -0.00508423, -0.00635487,  0.00026353,  0.00357465,
       -0.00787182, -0.00251322, -0.00391168, -0.01214969, -0.01154874,
       -0.00332629, -0.00196435, -0.00051089,  0.0015515 , -0.00555801,
       -0.00501047,  0.01398455, -0.00114764,  0.00663001,  0.00896661,
       -0.00098225,  0.00453457, -0.00247772, -0.00804651,  0.00502003,
       -0.0067461 ,  0.0064368 , -0.00588753,  0.01086427, -0.00908087,
        0.00459908, -0.00802532,  0.00100556,  0.00544896,  0.00040345,
       -0.00018507,  0.00644288,  0.01032001,  0.01256949,  0.00302291,
       -0.00050707, -0.01364416,  0.00186482, -0.00200181,  0.00

In [ ]:
model.get_word_vector("the")

array([ 0.02137857,  0.01676384,  0.04503896, -0.0524342 ,  0.06966472,
        0.02554192, -0.02811272, -0.0125381 ,  0.03379885,  0.022515  ,
       -0.00410887, -0.043385  ,  0.00548688,  0.01112284, -0.05795665,
       -0.02040619,  0.03973591,  0.05103175, -0.05173127, -0.02768937,
       -0.01450215,  0.06939985,  0.02702775,  0.00922399, -0.01715516,
       -0.07961302, -0.03875691, -0.03920429, -0.00676623,  0.02291585,
       -0.04950753, -0.02018066, -0.02099022, -0.07745673, -0.09214057,
       -0.02726264, -0.0315864 ,  0.0202981 ,  0.0233455 , -0.03360463,
       -0.04834947,  0.10499092, -0.00102818,  0.05098762,  0.05997887,
        0.01068992,  0.05072624, -0.01362669, -0.04072247,  0.0327213 ,
       -0.03744594,  0.03770958, -0.02543122,  0.06992245, -0.07166345,
        0.03161179, -0.05292026, -0.00376694,  0.04681172, -0.00822764,
        0.0037593 ,  0.05232479,  0.07301264,  0.10612043,  0.01843838,
       -0.00348117, -0.08782569,  0.00184562, -0.01502951, -0.02

In [ ]:
model.get_nearest_neighbors('green')

[(0.3940635025501251, 'hollywood'),
 (0.3803780674934387, 'pop'),
 (0.37719613313674927, 'this'),
 (0.3761516511440277, 'olivia'),
 (0.3665911555290222, 'kevin'),
 (0.3632358908653259, 'young'),
 (0.3584491014480591, 'up'),
 (0.35696443915367126, 'my'),
 (0.35505664348602295, 'birthday'),
 (0.3540915846824646, 'reveals')]

Will be attempting vectors here.

In [ ]:
!pip install fasttext
#!pip install unzip
#!pip install gunzip
#!gunzip cc.en.300.bin.gz  #common issue: https://github.com/facebookresearch/fastText/issues/1034
import fasttext as ft

#!pip3 install pyfasttext
#from pyfasttext import FastText

# Loading model for English
modelog = ft.load_model(r"C:\Users\salon\Downloads\cc.en.300.bin") 
 
# Getting word vectors for 'one' and 'two'.
one = modelog.get_word_vector('one')
two = modelog.get_word_vector('two')

# Getting the sentence vector for the sentence "one two" in Finnish.
one_two = modelog.get_sentence_vector('one two')
one_two_avg = (one + two) / 2

ValueError: ignored

In [ ]:
#Open the dataset, and for each title, get the word vector as follows: 
vector = ft.get_word_vector(text)#<DOUBTS: Does this embed only words (as opposed to sentences or sequences of words)... If it embeds only words then the average vector that represents the sentence is the dimension-wise average of all vectors of its words? What does it go if you give it an out-of-vocabulary word? (Does it give an error or a "blank" vector?)
#Save the word vectors in a dataframe that has 2 fields: x (word vector) and y (label as fake_news or not)
#Then take the dataframe that you have and train an XGBoost model over it.